## Grammatical Evolution applied to NN

In this notebook we try to develop a very simple example of GP applied to NNs with just one layer.

In [1]:
from random import random, randint, seed
from statistics import mean
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

import numpy as np
import math
import copy

The rationale behind the design of this grammar is the evolution of networks composed of one hidden-layer, where only the hidden-neurons as well as the weights of the connections
from the input and to the output neurons are evolved.

```
<sigexpr> ::= <node>
            | <node> + <sigexpr>
<node> ::= <weight> * sig(<sum> + <bias>)
<sum> ::= <weight> * <features>
        | <weight> * <features> + <sum>
<features> ::= x1
            | : : :
            | xn
<weight> ::= <number>
<bias> ::= <number>
<number> ::= <digit>.<digit><digit>
            | –<digit>.<digit><digit>
<digit> ::= 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9

```

In [42]:
n = 6
XLIST = np.random.rand(n) # initial random input of lenght n - features

def add(x,y): return x+y
def mul(x,y): return x*y
def sig(x): return 1 / (1 + math.exp(-x))

FUNCTIONS = [add, mul, sig]

In [58]:
class GP_MLP:
    def __init__(self, data = None, left = None, right = None, weights = None):
        self.data  = data
        self.left  = left
        self.right = right
        self.genotype = []
        self.weights = []
        self.biases = []

    def build_mlp(self, branch, grow, max_depth, depth= 0, parent=None):
        if random() > 0.5:
            parent.genotype.append(0)
            branch.build_node(branch, grow, max_depth, depth = 0, parent=parent)
        else:
            parent.genotype.append(1)
            branch.data = add
            branch.left = GP_MLP()
            branch.build_node(branch.left, grow, max_depth, depth = 1, parent=parent)
            branch.right = GP_MLP()
            branch.build_mlp(branch.right, grow, max_depth, depth = 1, parent=parent)


    def build_node(self, branch, grow, max_depth, depth = 0, parent = None):
        parent.weights.append(0)
        branch.data = mul
        branch.left = GP_MLP()      
        w = np.random.rand()
        branch.left.data = w
        parent.weights.append(w)

        branch.right = GP_MLP()
        branch.right.data = sig
        branch.right.left = GP_MLP()
        branch.right.left.data = add
        branch.right.left.left = GP_MLP()
        branch.add_sum(branch.right.left.left, True, max_depth, 1, branch.genotype)
        branch.right.left.right = GP_MLP()
        b = np.random.rand()
        branch.right.left.right.data = b
        parent.biases.append(b)
            
    def add_sum(self, branch, grow, max_depth, depth = 0, parent = None): # create random tree using either grow or full method

        if random() > 0.6 or depth >= max_depth: 
            branch.data = mul
            branch.add_sum_rec(branch)
            parent.append(0)
        else:
            branch.data = add
            branch.left = GP_MLP()          
            branch.left.data = mul
            
            branch.left.add_sum_rec(branch.left)

            branch.right = GP_MLP()
            branch.right.add_sum(branch, grow, max_depth, depth = depth + 1, parent=parent)
            parent.append(1)
        
        

    def add_sum_rec(self, branch):
        branch.left = GP_MLP() 
        w = np.random.rand()
        branch.left.data = w
        self.weights.append(w)
        branch.right = GP_MLP()
        branch.right.data = XLIST[randint(0, len(XLIST)-1)]

    def compute_tree(self): 
        if self.data == sig:
            return self.data(self.left.compute_tree())
        elif (self.data in FUNCTIONS): 
            return self.data(self.left.compute_tree(), self.right.compute_tree()) # perform the corresponding operation
        else:
            return self.data

    def node_label(self): # string label
        if (self.data in FUNCTIONS):
            return self.data.__name__
        else: 
            return str(round(self.data,3))
    
    def print_tree(self, prefix = ""): # textual printout
        print("%s%s" % (prefix, self.node_label()))        
        if self.left:  self.left.print_tree (prefix + "   ")
        if self.right: self.right.print_tree(prefix + "   ")

    def print_genotype(self):
        print(self.genotype)

    def get_genotype(self):
        return self.genotype

    def get_weights(self):
        return self.weights

    def get_biases(self):
        return self.biases
              

In [65]:
t = GP_MLP()
t.build_mlp(t, grow = True, max_depth = 3, depth= 0, parent = t) 
t.print_genotype()
t.print_tree()

[1, 1, 1, 0]
add
   mul
      0.089
      sig
         add
            mul
               0.809
               0.675
            0.157
   add
      mul
         0.459
         sig
            add
               mul
                  0.816
                  0.488
               0.693
      add
         mul
            0.911
            sig
               add
                  mul
                     0.337
                     0.257
                  0.685
         mul
            0.582
            sig
               add
                  mul
                     0.689
                     0.675
                  0.703


In [45]:
t.compute_tree()

0.6792044672257038

In [46]:
t.get_weights()

[0, 0.9872339111023377]

In [47]:
t.get_biases()

[0.25432475387655984]